# 🧪 Teste de Geração de Imagens - SYNTH_IMAGE

Este notebook testa a geração de imagens sintéticas de pastagens brasileiras no Google Colab.

---

## 📦 Instalação e Configuração

In [ ]:
# Instalar dependências faltantes
!pip install controlnet-aux ultralytics xformers --upgrade --quiet

# Verificar instalação GPU
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA disponível: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM disponível: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 🔧 Importações

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Configurar device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando device: {device}")

## 🤖 Carregamento do Modelo

In [ ]:
# Carregar modelo Stable Diffusion
model_id = "runwayml/stable-diffusion-v1-5"

print("Carregando modelo Stable Diffusion...")
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    safety_checker=None,
    requires_safety_checker=False
).to(device)

# Usar scheduler mais rápido
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

# Otimização para Colab (versão compatível)
if device == "cuda":
    try:
        pipe.enable_model_cpu_offload()
        print("CPU offload habilitado")
    except:
        print("CPU offload não disponível")

print("Modelo carregado com sucesso!")

## 🎨 Geração de Imagens de Teste

In [ ]:
# Prompts para pastagens brasileiras
prompts = [
    "aerial view of brazilian pasture with cattle, green grass, realistic, high quality",
    "tropical grassland in brazil, savanna landscape, cattle grazing, blue sky, photorealistic",
    "brazilian ranch pasture, green fields, farm animals, countryside, detailed",
    "cerrado landscape brazil, grass field, scattered trees, natural lighting"
]

# Parâmetros de geração
generator = torch.Generator(device=device).manual_seed(42)

# Gerar imagens
images = []
for i, prompt in enumerate(prompts[:2]):  # Gerar apenas 2 para teste
    print(f"Gerando imagem {i+1}/2: {prompt[:50]}...")
    
    image = pipe(
        prompt=prompt,
        negative_prompt="low quality, blurry, distorted, ugly, bad anatomy",
        num_inference_steps=20,  # Reduzido para velocidade
        guidance_scale=7.5,
        width=512,
        height=512,
        generator=generator
    ).images[0]
    
    images.append(image)
    
print("Imagens geradas com sucesso!")

## 📸 Visualização dos Resultados

In [ ]:
# Exibir imagens geradas
fig, axes = plt.subplots(1, 2, figsize=(15, 7))

for i, (image, prompt) in enumerate(zip(images, prompts[:2])):
    axes[i].imshow(image)
    axes[i].set_title(f"Imagem {i+1}\n{prompt[:40]}...", fontsize=10)
    axes[i].axis('off')

plt.tight_layout()
plt.show()

# Salvar imagens
for i, image in enumerate(images):
    image.save(f"pastagem_brasileira_{i+1}.png")
    print(f"Imagem {i+1} salva como: pastagem_brasileira_{i+1}.png")

## 🎯 Teste ControlNet (Opcional)

In [ ]:
# Testar ControlNet se disponível
try:
    from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
    from controlnet_aux import CannyDetector
    import cv2
    
    print("✅ ControlNet disponível!")
    
    # Criar uma imagem de borda simples para teste
    canny = CannyDetector()
    
    # Usar primeira imagem gerada como base
    if images:
        canny_image = canny(images[0])
        
        # Exibir resultado
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(images[0])
        plt.title("Imagem Original")
        plt.axis('off')
        
        plt.subplot(1, 2, 2)
        plt.imshow(canny_image, cmap='gray')
        plt.title("Bordas Canny")
        plt.axis('off')
        plt.show()
        
        print("ControlNet testado com sucesso!")
    
except ImportError as e:
    print(f"❌ ControlNet não disponível: {e}")
except Exception as e:
    print(f"⚠️ Erro no teste ControlNet: {e}")

## 🎯 Teste YOLO (Opcional)

In [ ]:
# Testar YOLO com Segmentação de Gramíneas
try:
    from ultralytics import YOLO
    import cv2
    import numpy as np
    
    print("✅ YOLO disponível!")
    
    # Carregar modelo de segmentação YOLO (YOLOv8 segmentation)
    print("Carregando modelo YOLOv8 para segmentação...")
    model = YOLO('yolov8n-seg.pt')  # Modelo de segmentação
    
    # Testar segmentação na primeira imagem
    if images:
        print("Realizando segmentação da imagem...")
        results = model(images[0])
        
        # Obter resultado
        result = results[0]
        
        # Verificar se há máscaras de segmentação
        if result.masks is not None:
            # Converter imagem para numpy
            img_array = np.array(images[0])
            
            # Criar visualização
            fig, axes = plt.subplots(2, 2, figsize=(15, 12))
            
            # 1. Imagem original
            axes[0, 0].imshow(images[0])
            axes[0, 0].set_title("Imagem Original")
            axes[0, 0].axis('off')
            
            # 2. Detecções com caixas
            annotated_img = result.plot()
            axes[0, 1].imshow(cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB))
            axes[0, 1].set_title("Detecções YOLO")
            axes[0, 1].axis('off')
            
            # 3. Máscaras de segmentação combinadas
            masks = result.masks.data.cpu().numpy()
            combined_mask = np.zeros(masks[0].shape)
            
            # Classes relacionadas a gramíneas/vegetação
            grass_classes = []
            for i, (mask, cls, conf) in enumerate(zip(masks, result.boxes.cls, result.boxes.conf)):
                class_name = model.names[int(cls)]
                confidence = float(conf)
                
                # Filtrar classes relacionadas à vegetação/gramíneas
                vegetation_keywords = ['grass', 'plant', 'vegetation', 'field', 'lawn']
                if any(keyword in class_name.lower() for keyword in vegetation_keywords) or confidence > 0.3:
                    combined_mask += mask
                    grass_classes.append((class_name, confidence))
                    print(f"Detectado: {class_name} (confiança: {confidence:.2f})")
            
            # Normalizar máscara
            combined_mask = np.clip(combined_mask, 0, 1)
            
            axes[1, 0].imshow(combined_mask, cmap='green', alpha=0.8)
            axes[1, 0].set_title("Máscara de Gramíneas/Vegetação")
            axes[1, 0].axis('off')
            
            # 4. Overlay da máscara na imagem original
            overlay = img_array.copy()
            green_mask = np.zeros_like(img_array)
            green_mask[:, :, 1] = combined_mask * 255  # Canal verde
            
            # Aplicar overlay
            alpha = 0.4
            overlay_result = cv2.addWeighted(img_array, 1-alpha, green_mask, alpha, 0)
            
            axes[1, 1].imshow(overlay_result)
            axes[1, 1].set_title("Overlay: Gramíneas Detectadas")
            axes[1, 1].axis('off')
            
            plt.tight_layout()
            plt.show()
            
            # Estatísticas da segmentação
            total_pixels = combined_mask.shape[0] * combined_mask.shape[1]
            grass_pixels = np.sum(combined_mask > 0)
            grass_percentage = (grass_pixels / total_pixels) * 100
            
            print(f"\n📊 Análise de Gramíneas:")
            print(f"Área total da imagem: {total_pixels} pixels")
            print(f"Área de gramíneas detectada: {grass_pixels} pixels")
            print(f"Percentual de gramíneas: {grass_percentage:.1f}%")
            
            if grass_classes:
                print(f"Classes detectadas: {grass_classes}")
            
        else:
            # Fallback: detecção simples se não houver máscaras
            annotated_frame = result.plot()
            plt.figure(figsize=(10, 8))
            plt.imshow(cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB))
            plt.title("Detecções YOLO (sem segmentação)")
            plt.axis('off')
            plt.show()
            
            print("⚠️ Modelo não retornou máscaras de segmentação")
        
        print("✅ Segmentação YOLO concluída!")
    
except ImportError as e:
    print(f"❌ YOLO não disponível: {e}")
except Exception as e:
    print(f"⚠️ Erro na segmentação YOLO: {e}")
    import traceback
    traceback.print_exc()

## 📊 Métricas de Performance

## 🌱 Segmentação Avançada de Gramíneas

In [ ]:
# Segmentação específica de gramíneas usando técnicas de cor e textura
def segment_grass_advanced(image):
    """
    Segmentação avançada de gramíneas baseada em cor, textura e características visuais
    """
    # Converter para numpy array
    img_array = np.array(image)
    
    # Converter para HSV para melhor segmentação de cores
    hsv = cv2.cvtColor(img_array, cv2.COLOR_RGB2HSV)
    
    # Definir ranges de cor para gramíneas (tons de verde)
    # Verde claro a escuro, considerando diferentes condições de iluminação
    lower_green1 = np.array([35, 40, 40])   # Verde claro
    upper_green1 = np.array([85, 255, 255])
    
    lower_green2 = np.array([25, 30, 30])   # Verde mais amarelado (grama seca)
    upper_green2 = np.array([95, 255, 180])
    
    # Criar máscaras de cor
    mask_green1 = cv2.inRange(hsv, lower_green1, upper_green1)
    mask_green2 = cv2.inRange(hsv, lower_green2, upper_green2)
    grass_mask = cv2.bitwise_or(mask_green1, mask_green2)
    
    # Operações morfológicas para refinar a máscara
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    grass_mask = cv2.morphologyEx(grass_mask, cv2.MORPH_CLOSE, kernel)
    grass_mask = cv2.morphologyEx(grass_mask, cv2.MORPH_OPEN, kernel)
    
    # Filtrar por área (remover pequenos ruídos)
    contours, _ = cv2.findContours(grass_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    filtered_mask = np.zeros_like(grass_mask)
    
    min_area = 100  # Área mínima para considerar como gramínea
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > min_area:
            cv2.fillPoly(filtered_mask, [contour], 255)
    
    return filtered_mask

# Aplicar segmentação avançada
if images:
    print("🌱 Aplicando segmentação avançada de gramíneas...")
    
    # Segmentar primeira imagem
    grass_mask = segment_grass_advanced(images[0])
    
    # Criar visualização comparativa
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # Imagem original
    axes[0, 0].imshow(images[0])
    axes[0, 0].set_title("Imagem Original")
    axes[0, 0].axis('off')
    
    # Máscara de gramíneas
    axes[0, 1].imshow(grass_mask, cmap='Greens')
    axes[0, 1].set_title("Máscara de Gramíneas")
    axes[0, 1].axis('off')
    
    # Overlay colorido
    img_array = np.array(images[0])
    overlay = img_array.copy()
    
    # Aplicar cor verde nas áreas de gramínea
    overlay[grass_mask > 0] = [0, 255, 0]  # Verde puro
    blended = cv2.addWeighted(img_array, 0.7, overlay, 0.3, 0)
    
    axes[0, 2].imshow(blended)
    axes[0, 2].set_title("Overlay: Gramíneas Destacadas")
    axes[0, 2].axis('off')
    
    # Análise por componentes de cor HSV
    hsv = cv2.cvtColor(img_array, cv2.COLOR_RGB2HSV)
    
    axes[1, 0].imshow(hsv[:,:,0], cmap='hsv')
    axes[1, 0].set_title("Canal H (Matiz)")
    axes[1, 0].axis('off')
    
    axes[1, 1].imshow(hsv[:,:,1], cmap='gray')
    axes[1, 1].set_title("Canal S (Saturação)")
    axes[1, 1].axis('off')
    
    axes[1, 2].imshow(hsv[:,:,2], cmap='gray')
    axes[1, 2].set_title("Canal V (Valor)")
    axes[1, 2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Estatísticas detalhadas
    total_pixels = grass_mask.shape[0] * grass_mask.shape[1]
    grass_pixels = np.sum(grass_mask > 0)
    grass_percentage = (grass_pixels / total_pixels) * 100
    
    print(f"\n📊 Análise Detalhada de Gramíneas:")
    print(f"Resolução da imagem: {grass_mask.shape[1]}x{grass_mask.shape[0]}")
    print(f"Total de pixels: {total_pixels:,}")
    print(f"Pixels de gramínea detectados: {grass_pixels:,}")
    print(f"Cobertura de gramíneas: {grass_percentage:.2f}%")
    print(f"Área de gramíneas: ~{(grass_pixels * 0.01):.0f} m² (estimativa)")
    
    # Análise de qualidade da pastagem
    if grass_percentage > 70:
        quality = "Excelente 🟢"
    elif grass_percentage > 50:
        quality = "Boa 🟡" 
    elif grass_percentage > 30:
        quality = "Regular 🟠"
    else:
        quality = "Baixa 🔴"
    
    print(f"Qualidade da pastagem: {quality}")
else:
    print("❌ Nenhuma imagem disponível para segmentação")

In [ ]:
# Informações de memória
if torch.cuda.is_available():
    print("📊 Uso de Memória GPU:")
    print(f"Alocada: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    print(f"Reservada: {torch.cuda.memory_reserved() / 1e9:.2f} GB")
    print(f"Máxima alocada: {torch.cuda.max_memory_allocated() / 1e9:.2f} GB")

# Verificar tamanhos das imagens
if images:
    print("\n📸 Informações das Imagens:")
    for i, image in enumerate(images):
        print(f"Imagem {i+1}: {image.size} - Modo: {image.mode}")

print("\n✅ Teste concluído com sucesso!")
print("\n🎯 Próximos passos:")
print("- Ajustar prompts para pastagens brasileiras específicas")
print("- Implementar pipeline completo com ControlNet")
print("- Integrar detecção YOLO para validação")
print("- Configurar dataset personalizado")